In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

from pprint import pprint

from functools import partial

import numpy as np
from sklearn.preprocessing import scale
import sklearn.metrics as metrics

import matplotlib.pyplot as plt
import matplotlib.animation as ani

import mlgrad.models as models
import mlgrad.regr as regr
import mlgrad.cls as cls
import mlgrad.pca as pca
import mlgrad.plots as plots
import mlgrad.funcs as funcs
import mlgrad.funcs2 as funcs2
import mlgrad.loss as loss
import mlgrad.gd as gd
import mlgrad.smooth as smooth
import mlgrad.inventory as inventory
from mlgrad.af import averaging_function
import mlgrad.avragg as avragg

import rampy
import pybaselines
import random

from irsa import io
from irsa.preprocess import despike, utils
import irsa.widgets as widgets
import irsa.spectra as spectra
import irsa.io as io

import scipy.special as special

import copy

import ipywidgets
from IPython.display import display

import sys
print(sys.version)

%matplotlib widget
plt.ioff()

3.13.2 (main, Mar 13 2025, 14:29:07) [GCC 14.2.0]


In [2]:
# path = f"/home/intellimath/repos/spectra/bacteria1024"
# path = f"/home/intellimath/repos/spectra/bacteria25_7"
# path = f"/home/intellimath/repos/spectra/mechanical_lysis"
# path = f"/home/intellimath/repos/spectra/thermal_lysis"
path = f"/home/intellimath/repos/spectra/7_4"
options = {}
dd = {}
# widgets.load_spectras(path, dd, options)

In [3]:
# dd = io.load_spectras(path, {"вид_бактерий":"STR", "отсечки_по_молекулярной_массе":"less30more10"})
dd = io.load_spectra(path, {"вид_бактерий":["SA", "SS", "SH"], 
                            "отсечки_по_молекулярной_массе": ["more100"],})
# dd = io.load_spectras(path, {"штамм_бактерий":["36", "95"], #"A", "B", "C", "D"], 
#                              "отсечки_по_молекулярной_массе": "less100",
#                              "тип_измерения_спектров":"SE"})
# dd = io.load_spectras(path, {})
# print(list(dd.keys()))

d_keys = list(sorted(dd.keys()))
sel_i = 0
sel = ipywidgets.Dropdown(options=d_keys,  description="Experiment:", rows=len(dd)+1)
sel.layout.width="50%"

/home/intellimath/repos/spectra/7_4/sources/7_4_SA_95_R_more100_3_H2O(HPLC)
7_4_SA_95_R_more100_3_H2O(HPLC) (1024,) (15, 1024) {'вид_бактерий': 'SA', 'отсечки_по_молекулярной_массе': 'more100'}
/home/intellimath/repos/spectra/7_4/sources/7_4_SH_15_more100_3_H2O(HPLC)
7_4_SH_15_more100_3_H2O(HPLC) (1024,) (15, 1024) {'вид_бактерий': 'SH', 'отсечки_по_молекулярной_массе': 'more100'}
/home/intellimath/repos/spectra/7_4/sources/7_4_SS_17_more100_3_H2O(HPLC)
7_4_SS_17_more100_3_H2O(HPLC) (1024,) (15, 1024) {'вид_бактерий': 'SS', 'отсечки_по_молекулярной_массе': 'more100'}


In [4]:
print(dd.keys())

dict_keys(['20250328_SA_95_R_more100_7_4_1_SE_no_date_', '20250328_SH_15_no_date_more100_7_4_1_SE_no_date_', '20250328_SS_17_no_date_more100_7_4_1_SE_no_date_'])


In [5]:
for i, key in enumerate(dd):
    entry1 = dd[key]
    entry1.allign_bottom()
    # entry1.scale_by_max(scale=10.0)
    entry1.crop(100)

In [6]:
@ipywidgets.interact(key=sel)
def plot_spectra_for_key(key):
    entry = dd[key]
    entry.plot_spectra()

interactive(children=(Dropdown(description='Experiment:', layout=Layout(width='50%'), options=('20250328_SA_95…

In [7]:
# @ipywidgets.interact(key=sel)
# def plot_sigma_mu_for_key(key):
#     entry = dd[key]
#     entry.plot_sigma_mu()

$$
\mu = \text{median}(X)
$$
$$
\sigma = \text{median}(|X-\mu|)
$$
$$
\text{modified-zscore}(X) = 0.6748 \frac{X - \mu}{\sigma}
$$

In [8]:
# @ipywidgets.interact(key=sel)
# def plots_zscore_for_key(key):
#     entry = dd[key]
#     entry.plot_zscore(kind="m-zscore")

In [9]:
# @ipywidgets.interact(key=sel)
# def plots_zscore_hist_for_key(key):
#     entry = dd[key]
#     entry.plot_zscore_hist()

In [10]:
@ipywidgets.interact(key=sel)
def check_spectra_for_key(key):
    entry = dd[key]
    entry.check_spectra()

interactive(children=(Dropdown(description='Experiment:', layout=Layout(width='50%'), options=('20250328_SA_95…

In [11]:
for i, key in enumerate(dd):
    entry = dd[key]
    entry.exclude_checked()    

In [12]:
# @ipywidgets.interact(key=sel)
# def plot_spectra_for_key(key):
#     entry = dd[key]
#     entry.plot_spectra()

In [12]:
d2 = spectra.SpectraCollection()
for i, key in enumerate(dd):
    entry2 = dd[key]
    d2[key] = entry2.robust_averaging(tau=3.5)

In [13]:
for key in d2:
    entry = d2[key]
    entry.allign_bottom()
    entry.scale_by_max(scale=1000.0)
    # entry.apply_func(y_func=np.log, x_func=np.log, b=1)


In [14]:
d2.save("/home/intellimath/work/spectra/spectra_SA_SS_SH", "series-average")

In [17]:
# @ipywidgets.interact(key=sel)
# def plot_spectra_robust_smoothing(key):
#     entry = d2[key]
#     entry.plot_spectra_with_robust_smoothing(tau=1.0, alpha=0.99)

interactive(children=(Dropdown(description='Experiment:', layout=Layout(width='50%'), options=('20250328_SA_95…